In [1]:
### http://deeplearning.net/tutorial/rnnslu.html ###
## https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/rnn_theano.py ##
#https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/train-theano.py#
#http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/#

import numpy as np
import theano
import theano.tensor as T

In [122]:
#-------------------------------------------------------#
## Start building the model ##
class RecurrentNet(object):
    def __init__(self,num_h=100,word_dim=8000,bptt_truncate=4):
        self.num_h = num_h
        self.word_dim = word_dim
        self.bptt_truncate = bptt_truncate
        # init network param Wih,Who,Whh
        Wih_val = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim),(num_h,word_dim))
        Whh_val = np.random.uniform(-np.sqrt(1./num_h), np.sqrt(1./num_h),(num_h,num_h))
        Who_val = np.random.uniform(-np.sqrt(1./num_h), np.sqrt(1./num_h),(word_dim,num_h))
        # set value to shared variables
        self.Wih = theano.shared(value=Wih_val.astype(theano.config.floatX),borrow=True,name='Wih')
        self.Whh = theano.shared(value=Whh_val.astype(theano.config.floatX),borrow=True,name='Whh')
        self.Who = theano.shared(value=Who_val.astype(theano.config.floatX),borrow=True,name='Who')
        # setup biases
        self.bh = theano.shared(np.zeros((num_h,1)).astype(theano.config.floatX),borrow=True, name='bh')
        self.bo = theano.shared(np.zeros((word_dim,1)).astype(theano.config.floatX),borrow=True, name='bo')
        # state of hidden layer/network
        state_val = np.zeros(shape=(100,1),dtype=theano.config.floatX)
        self.state = theano.shared(value=state_val,borrow=True,name='state')
        # cost
        cost = T.sum(T.nnet.categorical_crossentropy(o, y))
    
    # forward propagation
    def fstep(self,i):
        # given input i, calculate current state 
        self.state = T.tanh(T.dot(self.Whh,self.state) + T.dot(self.Wih,i) + self.bh)
        o = T.nnet.softmax(T.dot(self.Who,self.state) + self.bo)
        return o,np.argmax(o, axis=1)